# Network Preperation
This notebook proceses the network to get it ready for routing/matching/etc.

In [48]:
from pathlib import Path
import time
import geopandas as gpd
import pandas as pd
import numpy as np
import pickle
from shapely.ops import MultiLineString

import src.modeling_turns as modeling_turns

In [49]:
network_filepath = Path.home() / 'Documents/BikewaySimData/Projects/gdot/networks'

#bikewaysim_studyarea = gpd.read_file(Path.home()/'Documents/BikewaySimData/Data/Study Areas/bikewaysim_studyarea.geojson')

#import network links
#nodes = gpd.read_file(fp/'reconciled_network.gpkg',layer='nodes')
links = gpd.read_file(network_filepath/'reconciled_network.gpkg',layer='links_w_signals_elevation')
#links = gpd.read_file(network_filepath/'filtered.gpkg',layer='osm_links')

### Explore links with the same start/end node id and decide what to do with them
We could try to retain these, but at the moment it may make more sense to choose the shorter one and drop the other.

In [50]:
# import numpy as np
# test = np.array(links[['osm_A','osm_B']])#.sort(axis=0)
# duplicated = pd.DataFrame(np.sort(test)).duplicated(keep=False)
# duplicated_links = links[duplicated]
# duplicated_links.to_file(network_filepath/'duplicated_links.gpkg')

Binary Variables get turned to 1/0s, so gotta change back to true/false

In [51]:
links['oneway'] = links['oneway'] == "1"
links['length_ft'] = links.length

# Create new link variables

In [52]:
#vehicle seperation
links.loc[(links['pbl']==1) | (links['mu']==1),'vehicle_separation'] = 3
links.loc[links['bl'] == 1,'vehicle_separation'] = 2
links.loc[links['vehicle_separation'].isna(),'vehicle_separation'] = 1

### Create pseudo graph for modeling turns

In [53]:
links.reset_index(inplace=True)
links.rename(columns={'index':'linkid'},inplace=True)

In [ ]:
edges_df, pseudo_df, pseudo_G = modeling_turns.create_pseudo_dual_graph_linkid(links,'A','B','linkid','oneway')

In [54]:
edges_df, pseudo_df, pseudo_G = modeling_turns.create_pseudo_dual_graph(links,'A','B','linkid','oneway')

## Pick out desired attributes

In [55]:
attrs = ['linkid', 'osmid', 'link_type', 'name', 'highway',
       'bridge', 'tunnel', 'bl', 'pbl', 'mu','speedlimit_range_mph',
       'lanes_per_direction', 'up_grade', 'rise_m', 'down_grade', 'down_m', 'length_ft',
       'vehicle_separation']
edges_df = edges_df.merge(links[attrs],on='linkid',how='left')

## Deal with grade
Need to flip sign of grade for reverse links

In [56]:
edges_df['rise_m'] = np.where(edges_df['reverse_link'], edges_df['down_m'].abs(), edges_df['up_grade'])
edges_df['up_grade'] = np.where(edges_df['reverse_link'], edges_df['down_grade'].abs(), edges_df['up_grade'])
edges_df.drop(columns=['down_m','down_grade'],inplace=True)

## Turns and Signals

In [57]:
#add additional attributes needed for processing
source_links = links[['linkid','osmid','link_type','name','highway']]
target_links = links[['linkid','osmid','link_type','name','highway']]
source_links.columns = 'source_' + source_links.columns
target_links.columns = 'target_' + target_links.columns
pseudo_df = pseudo_df.merge(source_links,on='source_linkid',how='left')
pseudo_df = pseudo_df.merge(target_links,on='target_linkid',how='left')

## Turn Restrictions
Two types in OSM (represented as OSM relations):
- No (blank) turns
- Only this turn allowed

In [58]:
turn_restrictions = pd.read_csv(fp.parent/'osm_turn_restrictions.csv')
pseudo_df = pseudo_df.merge(turn_restrictions,left_on=['source_osmid','target_osmid'],right_on=['from_way_id','to_way_id'],how='left')
road_cond = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
no_restr = pseudo_df['type'] == 'no'
only_restr = pseudo_df['type'] == 'only'

#add a remove column
pseudo_df['remove'] = False

#remove the no turns
pseudo_df.loc[road_cond & no_restr,'remove'] = True

#for only, find all instances road_cond + from source and set to True
sources = set(turn_restrictions.loc[turn_restrictions['type']=='only','from_way_id'].tolist())
pseudo_df.loc[road_cond & pseudo_df['source_osmid'].isin(sources) & pseudo_df['type'].isna(),'remove'] = True

#Remove these turns and drop the added columns
print((pseudo_df['remove']==True).sum(),'turns removed')
pseudo_df = pseudo_df[pseudo_df['remove']==False]
pseudo_df.drop(columns=['relation_id', 'restriction', 'from_way_id',
       'to_way_id', 'type', 'remove'],inplace=True)

NameError: name 'fp' is not defined

# Deal with signals
Perform two merges and use the source/target reverse link columns to determine which signal ID to keep.
- For the source link, use signal_B if reverse == False else signal_A
- For the target link, use signal_A if reverse == False else signal_B

In [ ]:
source = pseudo_df[['source_linkid','source_reverse_link']].merge(links,left_on='source_linkid',right_on='linkid',how='left')
pseudo_df['source_signal'] = np.where(source['source_reverse_link'], source['signal_A'], source['signal_B'])

target = pseudo_df[['target_linkid','target_reverse_link']].merge(links,left_on='target_linkid',right_on='linkid',how='left')
pseudo_df['target_signal'] = np.where(target['target_reverse_link']==False, target['signal_A'], target['signal_B'])

## Identifying signalized/unsignalized turns
- Only look at roads for now
- Filter to left/right turns per source linkid per direction
- Take the highest road classification and assign it as the cross street road classification

In [ ]:
highway_order = {
    'trunk': 0,
    'trunk_link': 1,
    'primary': 2,
    'primary_link': 3,
    'secondary': 4,
    'secondary_link': 5,
    'tertiary': 6,
    'tertiary_link': 7,
    'unclassified': 8,
    'residential': 9
}
highway_order = pd.Series(highway_order)
highway_order = highway_order.reset_index()
highway_order.columns = ['highway','order']

In [ ]:
#add highway ranking based on the above
pseudo_df['target_highway_order'] = pseudo_df['target_highway'].map(highway_order.set_index('highway')['order'])
pseudo_df['source_highway_order'] = pseudo_df['source_highway'].map(highway_order.set_index('highway')['order'])

In [ ]:
#remove straight and uturn
cond1 = pseudo_df['turn_type'].isin(['left','right'])
#only road to road for now
cond2 = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
cross_streets = pseudo_df[cond1 & cond2]

#use groupby to find the max target_highway order
cross_streets = cross_streets.groupby(['source_linkid','source_A','source_B'])['target_highway_order'].min()
cross_streets.name = 'cross_street'

#add to main df
pseudo_df = pd.merge(pseudo_df,cross_streets,left_on=['source_linkid','source_A','source_B'],right_index=True,how='left')

#change numbers back to normal
pseudo_df['cross_street_order'] = pseudo_df['cross_street']
pseudo_df['cross_street'] = pseudo_df['cross_street'].map(highway_order.set_index('order')['highway'])

In [ ]:
signalized = pseudo_df['source_signal'] == pseudo_df['target_signal']
left_or_straight =  pseudo_df['turn_type'].isin(['left','straight'])
both_road = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
stressful_cross_street = pseudo_df['cross_street_order'] <= 5

pseudo_df.loc[signalized & left_or_straight & both_road,'signalized_left_straight'] = True
pseudo_df.loc[pseudo_df['signalized_left_straight'].isna(),'signalized_left_straight'] = False

pseudo_df.loc[-signalized & left_or_straight & both_road & stressful_cross_street,'unsignalized_left_straight_nonlocal'] = True
pseudo_df.loc[pseudo_df['unsignalized_left_straight_nonlocal'].isna(),'unsignalized_left_straight_nonlocal'] = False

#clean up
rem =  ['source_osmid', 'source_link_type', 'source_name',
       'source_highway', 'target_osmid', 'target_link_type', 'target_name',
       'target_highway', 'source_signal', 'target_signal',
       'target_highway_order', 'source_highway_order', 'cross_street',
       'cross_street_order']
pseudo_df.drop(columns=rem,inplace=True)

# Export for impedance calibration


In [ ]:
with (network_filepath.parent / 'impedance_calibration.pkl').open('wb') as fh:
    export = (edges_df,pseudo_df,pseudo_G)
    pickle.dump(export,fh)

## Add geometry to examine results in QGIS

In [125]:
#add geo
link_geo = dict(zip(links['linkid'],links['geometry']))
pseudo_df['src_geo'] = pseudo_df['source_linkid'].map(link_geo)
pseudo_df['trgt_geo'] = pseudo_df['target_linkid'].map(link_geo)
pseudo_df['geometry'] = pseudo_df[['src_geo','trgt_geo']].apply(lambda row: MultiLineString([row['src_geo'],row['trgt_geo']]),axis=1)

pseudo_df.drop(columns=['src_geo','trgt_geo'],inplace=True)
pseudo_df = gpd.GeoDataFrame(pseudo_df,crs=links.crs)

pseudo_df['source'] = pseudo_df['source'].astype(str)
pseudo_df['target'] = pseudo_df['target'].astype(str)

#check results (may need a smaller road network to test on)
pseudo_df.to_file(Path.home()/'Downloads/testing.gpkg',layer='cross_streets')